## TOM, 

I got it to work. The first 5 cells are needed and then everything after Combined Systems. Naturally you can remove everything used to view the results

In [1]:
import pandas as pd
import numpy as np

In [2]:
#load movies and ratings

movies = pd.read_csv('Data/movies_sml.csv')
ratings = pd.read_csv('Data/ratings_sml.csv')

In [3]:
# load movies with bag of words. Needed for content filtering

movies_bow = pd.read_csv('Data/movies_bow.csv')

In [4]:
#load ratings matrix.Needed for Collaborative filtering
ratings_matrix = pd.read_hdf('Data/ratings_mx.h5')

In [8]:
# load predictions
preds = pd.read_hdf('Data/preds_hdf2.h5')

###### test collab system first to see if any additional files are needed

In [6]:


def recommend_movies(predictions, userId, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userId - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userId)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

#     print (f"User {0} has already rated {1} movies. format(userID, user_full.shape[0]")
#     print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    rec_movies = recommendations['Title']
    already_rated = user_full['Title_x']
    return  already_rated, rec_movies

In [7]:
already_rated, predictions = recommend_movies(preds, 60, movies, ratings, 20)

In [8]:
predictions

423                       Schindler's List
215     Star Wars: Episode IV - A New Hope
293                           Forrest Gump
102                             Braveheart
6587                             Inception
6003                                    Up
5575                                WALL·E
1420                     Life Is Beautiful
1950                             Gladiator
8495                          Interstellar
249                 Léon: The Professional
485                                  Fargo
3110                          Finding Nemo
7820                      Django Unchained
8928                    The Imitation Game
5957                  Inglourious Basterds
5748                   Slumdog Millionaire
5059                           Ratatouille
2950                   Catch Me If You Can
9237                            Inside Out
Name: Title, dtype: object

###### test content system first to see if any additional files are needed

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Initializing and generating the count matrix 

count = CountVectorizer()
count_matrix = count.fit_transform(movies_bow['bag_of_words'])

In [11]:
# Generating the cosine similarity matrix

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [13]:
#map index to Title. This will be used in the function and is integral to 
# how we access the recommended movies

indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

#Create recommendation function

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:26]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_bow['Title'].iloc[movie_indices]

In [14]:
get_recommendations('Rumble in the Bronx')

11663                   Kung Fu Yoga
4129                       Who Am I?
3463                     Crime Story
670                       Supercop 2
690      Around the World in 80 Days
691      Around the World in 80 Days
1143                       Mr. Magoo
1416                       Rush Hour
1595                      Black Mask
2008                   Shanghai Noon
2440                     Rush Hour 2
4071                     City Hunter
4976                     Thunderbolt
5320                     Rush Hour 3
9890                    Dragon Blade
11352                      Skiptrace
11463                Railroad Tigers
1226                    Mr. Nice Guy
3052                Shanghai Knights
4397                        Gorgeous
4649                New Police Story
6441               Shinjuku Incident
6894              Little Big Soldier
4974                    Swordsman II
0                          Toy Story
Name: Title, dtype: object

#### Combined Systems

In [15]:
#Content-based Recommendations Sections
indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

indices

Title
Toy Story                                                       0
Jumanji                                                         1
Grumpier Old Men                                                2
Waiting to Exhale                                               3
Father of the Bride Part II                                     4
Heat                                                            5
Heat                                                            6
Sabrina                                                         7
Sabrina                                                         8
Tom and Huck                                                    9
Sudden Death                                                   10
GoldenEye                                                      11
Dracula: Dead and Loving It                                    12
Balto                                                          13
Nixon                                                          14
Cutt

In [16]:
idx = indices['Jumanji']

In [17]:
sim_scores = list(enumerate(cosine_sim[idx]))


In [18]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [143]:
sim_scores = sim_scores[1:26]

In [123]:
movie_indices = [i[0] for i in sim_scores]

In [124]:
content_results = pd.DataFrame(movies_bow['movieId'].iloc[movie_indices])


content_results 

,movieId
121,48593
412,500
601,765
1120,1702
1409,2253
1430,2297
1483,2431
1520,2501
1635,2718
1646,2759


In [54]:
# Collaborative Filtering Sections

# get user predictions

user_row_number = 60 # User ID starts at 1, not 0

sorted_user_predictions = pd.DataFrame(preds.iloc[user_row_number].sort_values(ascending=False).reset_index())

sorted_user_predictions.head()

,movieId,60
0,296,5.001825
1,2858,4.363025
2,608,3.931749
3,1193,3.701406
4,778,3.509410


In [157]:
# Get the movies the user originally rated
user_data = ratings[ratings.userId == user_row_number]

#combine the movies the user originally rated with the movies dataset and only keeping the movies
#the user rated

user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))

In [158]:
print(user_full.columns)

user_full = user_full[['userId', 'movieId', 'Title_x', 'rating', 'genres', 'Actors', 'Director', 'Plot', 'Poster']].\
            rename(columns = {'Title_x': 'Title'})


user_full.head()

Index(['userId', 'num_user_rated', 'movieId', 'Title_x', 'rating', 'Title_y',
       'genres', 'Actors', 'Director', 'Plot', 'Poster'],
      dtype='object')


,userId,movieId,Title,rating,genres,Actors,Director,Plot,Poster
0,60,4878,Donnie Darko,5.0,Drama|Mystery|Sci-Fi|Thriller,"Jake Gyllenhaal, Holmes Osborne, Maggie Gyllen...",Richard Kelly,A troubled teenager is plagued by visions of a...,https://m.media-amazon.com/images/M/MV5BZjZlZD...
52,60,8957,Saw,5.0,Horror|Mystery|Thriller,"Leigh Whannell, Cary Elwes, Danny Glover, Ken ...",James Wan,"Two strangers, who awaken in a room with no re...",https://m.media-amazon.com/images/M/MV5BMjE4MD...
32,60,714,Dead Man,5.0,Drama|Mystery|Western,"Johnny Depp, Gary Farmer, Crispin Glover, Lanc...",Jim Jarmusch,"On the run after murdering a man, accountant W...",https://m.media-amazon.com/images/M/MV5BYTJlZm...
33,60,1466,Donnie Brasco,5.0,Crime|Drama,"Al Pacino, Johnny Depp, Michael Madsen, Bruno ...",Mike Newell,An FBI undercover agent infiltrates the mob an...,https://m.media-amazon.com/images/M/MV5BYzMzMD...
22,60,2329,American History X,5.0,Crime|Drama,"Edward Norton, Edward Furlong, Beverly D'Angel...",Tony Kaye,A former neo-nazi skinhead tries to prevent hi...,https://m.media-amazon.com/images/M/MV5BZjA0MT...


In [136]:

num_recommendations = 1000

I noticed if the user doesn't watch anything similar to the movie inputted, we need to return a wide range of values from the sorted ratings. This is why num_recommendations is so high. 

One workaround is to limit num_recommendations to 100 and if there are no matches between content and collaborative results, set a message saying you will not like movies like 'movie name'

In [147]:
# Original Code from Collaborative Filter Function

# Recommend the highest predicted rating movies that the user hasn't seen yet.
movie_preds = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations]
)




In [148]:
movie_preds.shape

(1000, 9)

In [149]:
movie_preds.shape

(1000, 9)

In [150]:
# lookup movie predictions from content system from this dataframe

movie_recs = pd.merge(content_results, movie_preds_og, how='left', on='movieId').\
    sort_values('Predictions', ascending=False).head()

In [151]:
movie_recs

,movieId,Title,genres,Actors,Director,Plot,Poster,index,Predictions
11,3105,Awakenings,Drama|Mystery,"Robert De Niro, Robin Williams, Julie Kavner, ...",Penny Marshall,The victims of an encephalitis epidemic many y...,https://m.media-amazon.com/images/M/MV5BMjI5Nj...,302.0,0.185617
18,5528,One Hour Photo,Drama|Thriller,"Robin Williams, Connie Nielsen, Michael Vartan...",Mark Romanek,A mentally unstable photo developer targets a ...,https://m.media-amazon.com/images/M/MV5BYWVkMj...,365.0,0.156833
2,765,Jack,Comedy|Drama,"Robin Williams, Diane Lane, Brian Kerwin, Jenn...",Francis Ford Coppola,Because of an unusual aging disorder that has ...,https://m.media-amazon.com/images/M/MV5BZDYyYz...,439.0,0.129562
12,3156,Bicentennial Man,Drama|Romance|Sci-Fi,"Robin Williams, Embeth Davidtz, Sam Neill, Oli...",Chris Columbus,An android endeavors to become human as he gra...,https://m.media-amazon.com/images/M/MV5BYTU4Nz...,546.0,0.105328
6,2431,Patch Adams,Comedy|Drama,"Robin Williams, Daniel London, Monica Potter, ...",Tom Shadyac,"The true story of a heroic man, Hunter ""Patch""...",https://m.media-amazon.com/images/M/MV5BNzM4Y2...,628.0,0.088331
